In [0]:
-- ==============================================
-- GOLD / BI: Vista base para Power BI
-- Nombre: airbnb.gold.feature_bi_listing_performance
-- Objetivo: Fuente única para tableros (Import o DirectQuery)
-- Dependencias: silver.listings, silver.hosts, silver.reviews, silver.listing_amenities
-- ==============================================

CREATE SCHEMA IF NOT EXISTS airbnb.gold;

CREATE OR REPLACE VIEW airbnb.gold.feature_bi_listing_performance AS
WITH amen AS (
  SELECT
    bk_listing_id,
    COUNT(*) AS num_amenities
  FROM airbnb.silver.listing_amenities
  GROUP BY bk_listing_id
),
rev AS (
  SELECT
    bk_listing_id,
    COUNT(*)               AS total_reviews,
    MAX(review_date)       AS last_review_date
  FROM airbnb.silver.reviews
  GROUP BY bk_listing_id
)
SELECT
  -- Identidad
  l.bk_listing_id                                  AS listing_id,
  h.host_name,

  -- Dimensiones
  l.country,
  l.property_type,
  l.room_type,

  -- Métricas clave
  TRY_CAST(l.review_scores_rating AS INT)          AS avg_review_rating,
  COALESCE(r.total_reviews, 0)                     AS total_reviews,
  TRY_CAST(l.price AS DECIMAL(10,2))               AS avg_price_per_night,
  r.last_review_date,
  COALESCE(a.num_amenities, 0)                     AS num_amenities

FROM airbnb.silver.listings l
LEFT JOIN airbnb.silver.hosts h
  ON h.host_sk = l.host_sk       -- si prefieres negocio: h.bk_host_id = l.bk_host_id
LEFT JOIN amen a
  ON a.bk_listing_id = l.bk_listing_id
LEFT JOIN rev r
  ON r.bk_listing_id = l.bk_listing_id
WHERE l.price IS NOT NULL
  AND TRY_CAST(l.price AS DECIMAL(10,2)) > 0;
